# Here we want to convert data into the format needed for annotations

In [1]:
import pandas as pd
import re
import numpy as np
import glob
import csv
import hashlib

In [2]:
def main(path, destination):
    df = pd.DataFrame()
    batch = 1
    with open('movie_data_to_upload.csv', 'w+', newline='') as csvfile:
        fieldnames = ['batch_id','source','id1','text1','id2','text2','id3','text3','id4','text4','id5','text5','id6','text6','id7','text7','id8','text8','id9','text9','id10','text10','id11','text11','id12','text12','id13','text13','id14','text14','id15','text15','id16','text16','id17','text17','id18','text18','id19','text19','id20','text20']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for filename in glob.glob(path+'*.csv'):
            file = (filename.split("\\"))[1]
            m = hashlib.md5()
            m.update(file.encode('UTF-8'))
            movie_name_hex = int(str(int(m.hexdigest(), 16))[:10])
            
            df['batch_id'] =  movie_name_hex+batch

            df['source'] = file

            movie_df = pd.read_csv(filename)
            movie_df['source'] = ''
            movie_df['batch_id'] = ''
            i=0
            while i<len(movie_df):
                movie_df_chunk = movie_df[i:i+20][['number_of_dialog','dialog']]
                
                movie_df[i:i+20]['source'] = str(file)
                movie_df[i:i+20]['batch_id'] = str(movie_name_hex+batch)
                
                batch_dict = {}
                batch_dict['batch_id'] = str(movie_name_hex+batch)
                batch_dict['source'] = str(file)
                
                j=1
                for index, row in movie_df_chunk.iterrows():
                    batch_dict['id'+str(j)] = row.number_of_dialog
                    batch_dict['text'+str(j)] = row.dialog
                    j+=1
                                    
                writer.writerow(batch_dict)
                batch+=1
                i+=20
            movie_df.to_csv(destination + '/' + file)

In [3]:
main(path="CSV_3_combined_dialogs/", destination='mturk_format')